## 1. Load Data

In [1]:
include("src/model/Polypeptide.jl")

X = from_pdb("1A3N", "A")
Y = from_pdb("1A3N", "B")
render_aligned(X, Y)

LoadError: SystemError: opening file "C:\\Users\\stefa\\Dropbox\\Jotun Hein - Part C Projects\\Manolache\\Code\\Evolutionary Toroidal Difussions\\TorusEvol\\src\\model\\Polypeptide.jl": No such file or directory

## 2. Prepare Evolutionary Regimes

In [ ]:
include("src/distributions/WrappedNormal.jl")
include("src/distributions/WrappedDiffusion.jl")

diff = WrappedDiffusion([0.0, 1.0], 1.0, 1.0, 4.0, 1.0, 1.0)
t = 1.0
x = rand(statdist(diff), 200)
y = rand.(transdist.(Ref(diff), Ref(t), eachcol(x)))

In [199]:
y

2×200000 Matrix{Float64}:
 -0.629356   0.440885  -0.37218   0.312746  …  0.820192  0.573438  0.318079
  0.285451  -0.506242   1.72645  -0.203946     1.86111   0.839222  0.224563

In [200]:
using Turing

@model function wnn(x)
    mean ~ filldist(Uniform(-π, π), 2)
    var ~ filldist(Exponential(1), 4)
    Θ = (mean, var..., 0.0)
    diff = WrappedDiffusion(Θ...)
    Threads.@threads for i ∈ axes(x, 2)
        x[:,i] .~ statdist(diff)
    end
end

@model function fun(x, y)
    t ~ Exponential(1)
    mean ~ filldist(Uniform(-π, π), 2)
    var ~ filldist(Exponential(1), 4)
    Θ = (mean, var..., 0.0)
    diff = WrappedDiffusion(Θ...)

    Threads.@threads for i ∈ axes(x, 2)
        x[:,i] .~ statdist(diff)
        y[:,i] .~ transdist(diff, t, x[:, i])
    end

end

fun (generic function with 2 methods)

In [ ]:
alg = HMC(0.05, 10, :mean, :var)
l=10
model = wnn(x[:, 1:l])
num_samples = 10
chain = sample(model, alg, num_samples)